# Transfer learning - Fine-tuning



## Fine Tuning

In [4]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ])

trainset = datasets.CIFAR10(root='dataCIFAR10',train=True, download=True,transform=transform)
testset = datasets.CIFAR10(root='data/CIFAR10',train=False, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting dataCIFAR10/cifar-10-python.tar.gz to dataCIFAR10


In [9]:
for images, labels in trainloader:
  print(images.size(), labels.size())
  break

torch.Size([64, 3, 224, 224]) torch.Size([64])


In [31]:
model = models.vgg16(pretrained=True)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [32]:
for param in model.parameters():
  param.requires_grad = False

In [33]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [34]:
for i in range(0,7):
  model.classifier[i].requires_grad = True

In [35]:
model.classifier[6] = nn.Sequential(
                      nn.Linear(4096,512),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(512,10),
                      nn.LogSoftmax(dim=1)
                      )


In [36]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=10, bias=True)
    (4): LogSoftmax(dim=1)
  )
)

In [37]:
model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [38]:
criterion = nn.NLLLoss()

In [39]:
from torch.optim import Adam

model = model.to(device)
optimizer = Adam(model.parameters())


## Training from the Fully Connected Network onwards

means, training all the layers of the classifiers from Sequential 0 to Sequential 6 (including the 4 layers of the sequential 6)

### Re-training the model

In [41]:
model = model.to(device)

#Warning:
#The optimizer uses the loss value to update the network weights.
#When we try to update the weights, we might get an error because
#some of the weights are frozen (yet, we ask to update them). 
#The solution for this is to configure the optmiser with the updatable
#parameters only
#optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

optimizer = Adam(model.parameters())

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/782) : Batch loss : 0.6034636497497559
Epoch(0/1 : Batch number(2/782) : Batch loss : 0.6579328775405884
Epoch(0/1 : Batch number(3/782) : Batch loss : 0.6273006200790405
Epoch(0/1 : Batch number(4/782) : Batch loss : 0.4412669837474823
Epoch(0/1 : Batch number(5/782) : Batch loss : 0.3914874196052551
Epoch(0/1 : Batch number(6/782) : Batch loss : 0.5907207727432251
Epoch(0/1 : Batch number(7/782) : Batch loss : 0.4576740264892578
Epoch(0/1 : Batch number(8/782) : Batch loss : 0.756644070148468
Epoch(0/1 : Batch number(9/782) : Batch loss : 0.6697240471839905
Epoch(0/1 : Batch number(10/782) : Batch loss : 0.7876107692718506
Epoch(0/1 : Batch number(11/782) : Batch loss : 0.8131794929504395
Epoch(0/1 : Batch number(12/782) : Batch loss : 0.5774235129356384
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.4808422029018402
Epoch(0/1 : Batch number(14/782) : Batch loss : 0.48167887330055237
Epoch(0/1 : Batch number(15/782) : Batch loss : 0.49850013852119446
Epo

### The accuracy of the model

In [46]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 80.3125% 


## Un-freezing & training on the LAST CNN block onwards

### Re-training the model

In [47]:
for i in range(24,31):
  model.features[i].requires_grad = True


In [21]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/782) : Batch loss : 0.37638935446739197
Epoch(0/1 : Batch number(2/782) : Batch loss : 0.4667692184448242
Epoch(0/1 : Batch number(3/782) : Batch loss : 0.4867028594017029
Epoch(0/1 : Batch number(4/782) : Batch loss : 0.7602155208587646
Epoch(0/1 : Batch number(5/782) : Batch loss : 0.46336594223976135
Epoch(0/1 : Batch number(6/782) : Batch loss : 0.48384594917297363
Epoch(0/1 : Batch number(7/782) : Batch loss : 0.6412556171417236
Epoch(0/1 : Batch number(8/782) : Batch loss : 0.3660600483417511
Epoch(0/1 : Batch number(9/782) : Batch loss : 0.6906344294548035
Epoch(0/1 : Batch number(10/782) : Batch loss : 0.6811375021934509
Epoch(0/1 : Batch number(11/782) : Batch loss : 0.30804815888404846
Epoch(0/1 : Batch number(12/782) : Batch loss : 0.35521185398101807
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.6848178505897522
Epoch(0/1 : Batch number(14/782) : Batch loss : 0.5894997715950012
Epoch(0/1 : Batch number(15/782) : Batch loss : 0.3391475677490234

### The accuracy of the model

In [22]:
model.to('cpu')

model.eval()
w!wget https://drive.switch.ch/index.php/s/sywP12y6Iuiq4ex/download!wget https://drive.switch.ch/index.php/s/sywP12y6Iuiq4ex/download!wget https://drive.switch.ch/index.php/s/sywP12y6Iuiq4ex/download!wget https://drive.switch.ch/index.php/s/sywP12y6Iuiq4ex/download!wget https://drive.switch.ch/index.php/s/sywP12y6Iuiq4ex/download!wget https://drive.switch.ch/index.php/s/sywP12y6Iuiq4ex/downloadith torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 83.4375% 


## Un-freezing & training on the LAST TWO CNN block onwards

### Re-training the model

In [0]:
for i in range(17,24):
  model.features[i].requires_grad = True

In [24]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/782) : Batch loss : 0.4675794839859009
Epoch(0/1 : Batch number(2/782) : Batch loss : 0.25580158829689026
Epoch(0/1 : Batch number(3/782) : Batch loss : 0.5276210904121399
Epoch(0/1 : Batch number(4/782) : Batch loss : 0.3741523027420044
Epoch(0/1 : Batch number(5/782) : Batch loss : 0.5076795220375061
Epoch(0/1 : Batch number(6/782) : Batch loss : 0.3216595947742462
Epoch(0/1 : Batch number(7/782) : Batch loss : 0.3571995198726654
Epoch(0/1 : Batch number(8/782) : Batch loss : 0.2978670001029968
Epoch(0/1 : Batch number(9/782) : Batch loss : 0.36628425121307373
Epoch(0/1 : Batch number(10/782) : Batch loss : 0.45307356119155884
Epoch(0/1 : Batch number(11/782) : Batch loss : 0.4280996024608612
Epoch(0/1 : Batch number(12/782) : Batch loss : 0.3617429733276367
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.34377187490463257
Epoch(0/1 : Batch number(14/782) : Batch loss : 0.4511021375656128
Epoch(0/1 : Batch number(15/782) : Batch loss : 0.3250049352645874


### The accuracy of the model

In [25]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 82.1875% 
